In [2]:
import numpy as np
import matplotlib.pyplot as plt

## Question (a): 1D interstitial diffusion

Given Table 2.1 in Porter & Easterling, for nitrogen diffusing interstitially in BCC iron, we have

$D_0 = 0.3mm^2s^{-1}, Q = 76.1kJ\ mol^{-1}$

According to equation (2.9) in Porter & Easterling, at 1000K,

$D = D_0 exp(-Q/RT) = 0.3 * exp(-76*10^3/(8.314*1000))\ mm^2s^{-1} = 32.15\mu m^2s^{-1}$

In [7]:
def interstitial_1D(D=32.15, 
                    init=np.array([1.0]+[0]*999),
                    time=1000,
                   ):
    '''
    Solve the 1D diffusion equation dc/dt = D * d^2c/dx^2 
    for interstitial diffusion. 
    The boundary conditions are fixed concentration at the surface
    and zero flux through the other end.
    
    Parameters
    ----------
    D : Diffusion coefficient, float, unit is micron^2/s. 
        Default to the diffusion coefficient of interstitial nitrogen
        diffusing into BCC iron at 1000K.
    init : Initial concentration profile, float numpy array, 
           length of the array is the thickness in microns,
           concentration unit is picomole/micron^3.
           Default to a step function, everywhere except the surface is 0.
           Compare: pure Fe got 14.1 picomole/micron^3.
    time : Duration of diffusion, unit is second, step=1s.
           Default to 1000s.
    '''
    for i in range(time):
        